In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import gc

In [2]:
# Initialize the dictionary to hold paths for each animal
image_paths = {}

# Base directory for your dataset
base_path = "C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/"

# List of animals to search for
animals = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Traverse the directory structure
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        # Check if the current directory contains the animal's name
        if animal in dirname:
            # Initialize the list if the animal is encountered for the first time
            if animal not in image_paths:
                image_paths[animal] = []
            # Add all image paths for the current animal
            for filename in filenames:
                image_paths[animal].append(os.path.join(dirname, filename))

# Print the paths for verification
for animal, paths in image_paths.items():
    print(f"{animal}: {len(paths)} images")

collie: 1028 images
dolphin: 946 images
elephant: 1038 images
fox: 664 images
giant+panda: 874 images
moose: 704 images
polar+bear: 868 images
rabbit: 1088 images
sheep: 1420 images
squirrel: 1200 images


In [3]:
for animal, paths in image_paths.items():
    print(f"{animal}: {paths}")

collie: ['C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10001.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10002.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10003.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10004.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10005.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10006.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10007.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10008.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/collie\\collie_10009.jpg', 'C:/Users/Melih/Coding/Bootcamp/AwA2-data/Anim

In [4]:
# Çıkış dizini (işlenmiş resimler için)
output_path = "C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/processed_images"
os.makedirs(output_path, exist_ok=True)

# Resim boyutu (model girişine göre)
image_size = (128, 128)  # Örnek: 128x128

# Dizin yapısını dolaşarak ilk 650 resmi seç
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        if animal in dirname:
            if animal not in image_paths:
                image_paths[animal] = []
            for filename in filenames:
                image_paths[animal].append(os.path.join(dirname, filename))

# Her sınıf için yalnızca ilk 650 resmi işleyin
for animal, paths in image_paths.items():
    selected_paths = paths[:650]
    animal_output_dir = os.path.join(output_path, animal)
    os.makedirs(animal_output_dir, exist_ok=True)

    for i, image_path in enumerate(selected_paths):
        try:
            # Resmi yükle
            img = cv2.imread(image_path)

            # Yeniden boyutlandır ve normalize et
            img_resized = cv2.resize(img, image_size)
            img_normalized = img_resized / 255.0

            # Yeni dosya yolunu oluştur
            output_file = os.path.join(animal_output_dir, f"{i+1}.jpg")

            # Normalize edilmiş resmi kaydet
            cv2.imwrite(output_file, (img_normalized * 255).astype("uint8"))
        except Exception as e:
            print(f"Error processing {image_path}: {e}")


            # Belleği temizle
            del img
            del manipulated_img
            gc.collect()  # Çöp toplama işlemi başlat


print("Resimler başarıyla işlendi ve kaydedildi.")

Resimler başarıyla işlendi ve kaydedildi.


In [5]:
from sklearn.model_selection import train_test_split

# Resimler ve etiketler
X = []
y = []

# Resimleri yükleyip etiketlerle birlikte X ve y listelerini oluşturma
for idx, (animal, paths) in enumerate(image_paths.items()):
    for path in paths:
        img = cv2.imread(path)
        img_resized = cv2.resize(img, image_size)
        X.append(img_resized)
        y.append(idx)

# NumPy dizilerine dönüştürme
X = np.array(X)
y = np.array(y)

# Eğitim ve test verisini ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [6]:
# CNN modelinin kurulması
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(animals), activation='softmax')
])

# Modeli derleme
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model özeti
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 128)      0

In [7]:
# Modeli eğitme
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
gc.collect()


Epoch 1/10
431/431 [==============================] - 11s 21ms/step - loss: 2.5010 - accuracy: 0.2497 - val_loss: 1.9613 - val_accuracy: 0.3003
Epoch 2/10
431/431 [==============================] - 9s 20ms/step - loss: 1.8833 - accuracy: 0.3268 - val_loss: 1.8230 - val_accuracy: 0.3332
Epoch 3/10
431/431 [==============================] - 8s 20ms/step - loss: 1.6955 - accuracy: 0.3899 - val_loss: 1.6515 - val_accuracy: 0.3978
Epoch 4/10
431/431 [==============================] - 8s 20ms/step - loss: 1.3870 - accuracy: 0.5036 - val_loss: 1.3957 - val_accuracy: 0.5261
Epoch 5/10
431/431 [==============================] - 9s 20ms/step - loss: 1.1176 - accuracy: 0.6077 - val_loss: 1.3029 - val_accuracy: 0.5933
Epoch 6/10
431/431 [==============================] - 8s 20ms/step - loss: 0.8990 - accuracy: 0.6890 - val_loss: 1.3289 - val_accuracy: 0.6170
Epoch 7/10
431/431 [==============================] - 8s 20ms/step - loss: 0.7241 - accuracy: 0.7567 - val_loss: 1.2415 - val_accuracy: 0.667

1462

In [8]:
# Modeli test etme
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")
gc.collect()

185/185 [==============================] - 1s 7ms/step - loss: 1.2268 - accuracy: 0.7570
Test accuracy: 0.7570363283157349


731

In [9]:
# Manipülasyon fonksiyonu
def get_manipulated_images(image):
    # Parlaklık artışı
    manipulated_img = cv2.convertScaleAbs(image, alpha=1.5, beta=30)
    
    # Beyaz dengesi uygulama (örneğin, sıcaklık değişikliği)
    manipulated_img = cv2.xphoto.createSimpleWB().balanceWhite(manipulated_img)

    # Kontrast artırma
    lab = cv2.cvtColor(manipulated_img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)
    lab = cv2.merge([l, a, b])
    manipulated_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    return manipulated_img

# Manipüle edilmiş görüntülerin kaydedileceği klasör
output_dir = "C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images"
os.makedirs(output_dir, exist_ok=True)

# Resimleri manipüle etme ve kaydetme fonksiyonu
def manipulate_and_save_images(input_dir, output_dir):
    for animal in os.listdir(input_dir):
        animal_dir = os.path.join(input_dir, animal)
        if os.path.isdir(animal_dir):
            output_animal_dir = os.path.join(output_dir, animal)
            os.makedirs(output_animal_dir, exist_ok=True)

            for image_file in os.listdir(animal_dir):
                image_path = os.path.join(animal_dir, image_file)
                if image_file.endswith(".jpg") or image_file.endswith(".png"):
                    # Resmi oku
                    img = cv2.imread(image_path)

                    # Görüntüyü manipüle et
                    manipulated_img = get_manipulated_images(img)

                    # Yeni dosya yolunu oluştur
                    output_image_path = os.path.join(output_animal_dir, image_file)

                    # Manipüle edilmiş resmi kaydet
                    cv2.imwrite(output_image_path, manipulated_img)
                    print(f"Manipulated image saved at: {output_image_path}")

# Manipüle etme işlemi
input_directory = "C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/processed_images"
manipulate_and_save_images(input_directory, output_dir)

Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images\collie\1.jpg
Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images\collie\10.jpg
Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images\collie\100.jpg
Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images\collie\101.jpg
Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images\collie\102.jpg
Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images\collie\103.jpg
Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images\collie\104.jpg
Manipulated image saved at: C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_i

In [10]:
from tensorflow.keras.preprocessing import image

# Manipüle edilmiş görüntülerin bulunduğu klasör
manipulated_images_dir = "C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images"

# Görüntü boyutlandırma
image_size = (128, 128)

# Resimleri yükle ve işleyin
def load_manipulated_images(manipulated_images_dir, image_size):
    X = []
    y = []
    animals = os.listdir(manipulated_images_dir)
    
    for animal in animals:
        animal_dir = os.path.join(manipulated_images_dir, animal)
        if os.path.isdir(animal_dir):
            label = animals.index(animal)  # Etiketleme
            for image_file in os.listdir(animal_dir):
                image_path = os.path.join(animal_dir, image_file)
                if image_file.endswith(".jpg") or image_file.endswith(".png"):
                    # Resmi yükleyin ve yeniden boyutlandırın
                    img = cv2.imread(image_path)
                    img = cv2.resize(img, image_size)
                    X.append(img)
                    y.append(label)
    
    X = np.array(X)
    y = np.array(y)
    
    # Normalize etme (0-255 aralığından 0-1 aralığına)
    X = X / 255.0
    
    return X, y

# Manipüle edilmiş verileri yükleyin
X_manipulated, y_manipulated = load_manipulated_images(manipulated_images_dir, image_size)


# Modelin manipüle edilmiş veriyle testi
test_loss, test_acc = model.evaluate(X_manipulated, y_manipulated)
print(f"Manipulated test accuracy: {test_acc}")

204/204 [==============================] - 2s 9ms/step - loss: 11.5586 - accuracy: 0.1300
Manipulated test accuracy: 0.12999999523162842


In [11]:
def get_wb_images(image):
    """
    Gray World algoritmasını uygulayarak renk sabitliği yapar.
    """
    # Her renk kanalının ortalamasını hesapla
    avg_b = np.mean(image[:, :, 0])  # Blue
    avg_g = np.mean(image[:, :, 1])  # Green
    avg_r = np.mean(image[:, :, 2])  # Red
    
    # Ortalama parlaklık
    avg_gray = (avg_b + avg_g + avg_r) / 3
    
    # Kanalları normalize et
    scale_b = avg_gray / avg_b
    scale_g = avg_gray / avg_g
    scale_r = avg_gray / avg_r

    # Görüntüyü normalize et
    image[:, :, 0] = np.clip(image[:, :, 0] * scale_b, 0, 255)
    image[:, :, 1] = np.clip(image[:, :, 1] * scale_g, 0, 255)
    image[:, :, 2] = np.clip(image[:, :, 2] * scale_r, 0, 255)

    return image.astype(np.uint8)


In [12]:
# Manipüle edilmiş resimlerin ve sonuçların dizinleri
manipulated_images_dir = "C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images"
output_dir = "C:/Users/Melih/Coding/Bootcamp/AwA2-data/Animals_with_Attributes2/wb_images"
os.makedirs(output_dir, exist_ok=True)

# Tüm manipüle edilmiş test setini renk sabitliği ile işle ve kaydet
for root, _, files in os.walk(manipulated_images_dir):
    for file in files:
        input_image_path = os.path.join(root, file)

        # Çıkış klasör yapısını korumak için alt klasörler oluştur
        relative_path = os.path.relpath(root, manipulated_images_dir)
        output_subdir = os.path.join(output_dir, relative_path)
        os.makedirs(output_subdir, exist_ok=True)
        
        output_image_path = os.path.join(output_subdir, file)

        # Görüntüyü yükle
        image = cv2.imread(input_image_path)
        if image is not None:
            # Gray World algoritması uygula
            wb_image = get_wb_images(image)
            # Sonucu kaydet
            cv2.imwrite(output_image_path, wb_image)

print("Renk sabitliği tüm manipüle edilmiş test setine uygulandı.")

Renk sabitliği tüm manipüle edilmiş test setine uygulandı.


In [13]:
def load_wb_images(wb_images_dir, image_size):
    X = []
    y = []
    for root, _, files in os.walk(wb_images_dir):
        for file in files:
            image_path = os.path.join(root, file)
            label = os.path.basename(root)  # Klasör adını etiket olarak al
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, image_size)
                X.append(image)
                y.append(animals.index(label))  # `animals` listesine göre etiketle
    return np.array(X), np.array(y)

# Kullanım
wb_X_test, wb_y_test = load_wb_images(output_dir, (128, 128))

In [14]:
# Normalize et
wb_X_test = wb_X_test / 255.0

# Modelle test et
test_loss, test_acc = model.evaluate(wb_X_test, wb_y_test)
print(f"White balance applied test accuracy: {test_acc}")

204/204 [==============================] - 2s 8ms/step - loss: 12.8543 - accuracy: 0.1008
White balance applied test accuracy: 0.10076922923326492
